In [20]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re

In [25]:
# read in csv file
all_cases = gpd.read_file('/Users/leahwallihan/Durham_school_planning/Res_Development')
all_cases.head()

,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppCode,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry
0,S1900405,FIN_EXE,2019-11-26,APP,2019-12-13,Longebach Recombination Survey,Recombine two lots into 1,BRYANTGR,MICHAELKL,APP,...,FIN_EXE,Exempt Plat,Michael Klein,Michael.Klein@durhamnc.gov,1,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2024162.97 857829.602)
1,HO1400256,PL_HOP,2014-05-06,APP,2014-05-06,Proserv Cleaner,12 Trescott Drive is the home office for the c...,MARIAHER,HSMITH,APP,...,PL_HOP,Home Occupation Permit,None,None,2,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2055130.547 807863.401)
2,S1700288,FIN_EXE,2017-12-11,APP,2018-01-16,Brightleaf at the Park Tract 7 Easement,None,JESSICADO,JESSICADO,APP,...,FIN_EXE,Exempt Plat,Jessica Dockery,Jessica.Dockery@durhamnc.gov,3,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2054037.745 800861.745)
3,HO1100444,PL_HOP,2011-07-29,APP,2011-07-29,Fam or Foe Ent,Home office for musician - no onsite shows or ...,DDOTY,DDOTY,APP,...,PL_HOP,Home Occupation Permit,Dennis Doty,Dennis.Doty@durhamnc.gov,4,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2038546.286 813065.084)
4,S1400114,FIN_MAJ,2014-06-24,APP,2015-07-29,Phillip and Gina Ellis Plat,Minor Subdivision,AMANDAHO,JENNIFERL,APP,...,FIN_MAJ,Final Plat,Jennifer Leaf,Jennifer.Leaf@durhamnc.gov,5,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2047560.449 902477.57)


In [27]:
# types = A_TYPE codes we want to keep
types = ['PL_MINSP', 'PL_SSP_SM', 'PL_SSM_SM2', 'PL_MJ_COA', 'D_REZ', 'PL_CPAA', 'PL_MINPP', 'PL_MAJSP', 'PL_MAJSUP', 'PL_PPA', 'PL_MAJPP'] 
# filter that only keeps cases of specified types
filter_cases_type = all_cases[all_cases['A_TYPE'].isin(types)]

In [29]:
# status = A_STATUS codes we want to keep
status = all_cases['A_STATUS'].unique()
status = status[~np.isin(status, ['WITH', 'VOID'])]
# filter that only keeps cases of specified status
filter_cases_status = filter_cases_type[filter_cases_type['A_STATUS'].isin(status)]

In [31]:
# filter_cases_status

,A_NUMBER,A_TYPE,A_DATE,A_STATUS,A_STATUS_D,A_PROJECT_,A_DESCRIPT,A_USER_ID,A_CASE_PLA,StatCode,...,AppCode,AppType,CasePlanne,EMAIL,ORIG_FID,CreationDa,Creator,EditDate,Editor,geometry
11,COA1700068,PL_MJ_COA,2017-07-07,APP,2017-09-15,1110 Wall Street,New Construction,KROSENBERG,KROSENBERG,APP,...,PL_MJ_COA,Major COA,Karla Rosenberg,Karla.Rosenberg@durhamnc.gov,12,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2033475.776 815203.548)
15,D1900246,PL_MINSP,2019-07-25,APP,2020-02-28,Tru by Hilton Hotel,"Construct 4 story, 98 room hotel, parking and ...",AMANDAHO,TREYFI,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,16,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2044848.401 775163.066)
36,Z1400003,D_REZ,2014-02-21,APP,2014-08-05,2919 Fayetteville Street,allow retail/commercial uses in existing struc...,JOANNEG,AWOLFF,APP,...,D_REZ,Zoning Map Change,Amy Wolff,Amy.Wolff@durhamnc.gov,37,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2027721.218 805744.938)
49,D1200040,PL_SSP_SM,2012-02-28,APP,2012-04-04,Duke Student Activity Center Walkways,Provide new asphalt path from campus drive to ...,JEANETTEBR,LHODGES,APP,...,PL_SSP_SM,Level 3 - Administrative Site Plan,None,None,50,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2023483.75 818605.827)
56,D2200035,PL_MINSP,2022-02-11,APP,2022-05-15,Alexander Industrial Park - Building 3,"Proposal of 1 story ,40,950 SF of warehouse wi...",COLERE,COLERE,APP,...,PL_MINSP,Level 4 - Administrative Site Plan,Cole Renigar,Cole.Renigar@durhamnc.gov,57,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2051101.02 788077.191)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21267,D2500108,PL_MINSP,2025-05-02,UN_RE,2025-05-02,Pine View Buildings,Storage shed sales.,ERIKA,TREYFI,UN_RE,...,PL_MINSP,Level 4 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21268,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2006810.3 804532.158)
21270,D2500109,PL_MINSP,2025-05-06,UN_RE,2025-05-06,Davis Park East Retail,"Three, one story multi-use buildings with asso...",VCHIRICHELLA,PEYTONBL,UN_RE,...,PL_MINSP,Level 4 - Administrative Site Plan,PEYTON BLANCHARD,PEYTON.BLANCHARD@DURHAMNC.GOV,21271,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2042796.175 774410.666)
21296,D2500117,PL_SSP_SM,2025-05-12,UN_RE,2025-05-12,Prime STQ at HUB RTP,Expansion of Building B at HUB RTP Horseshoe C...,MITRAGO,TREYFI,UN_RE,...,PL_SSP_SM,Level 3 - Administrative Site Plan,Trey Figueroa,Trey.Figueroa@durhamnc.gov,21297,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2040518.641 780054.743)
21308,D2500118,PL_MINSP,2025-05-15,UN_RE,2025-05-15,Yard Waste Repairs and Upgrades,Improve and upgrade existing compost facility.,FRANCISCOM,PEYTONBL,UN_RE,...,PL_MINSP,Level 4 - Administrative Site Plan,PEYTON BLANCHARD,PEYTON.BLANCHARD@DURHAMNC.GOV,21309,2025-05-21,gisproc_sys,2025-05-21,gisproc_sys,POINT (2043917.766 832427.332)


/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field A_DATE create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field A_STATUS_D create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field CreationDa create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field EditDate create as date field, though DateTime requested.
  ogr_write(


In [37]:
keywords = ['home', 'family', 'residen', 'mixed', 'mized', 'duplex', 'apartment', ' hous', 'condo', 'dwelling', 'tenant', 'affordable', 'units', 'townhouse']
pattern = '|'.join(keywords)

filtered = filter_cases_status[filter_cases_status['A_DESCRIPT'].str.contains(pattern, case=False, na=False)]

In [47]:
filtered.to_file(filename='res_key_filtered')

/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field A_DATE create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field A_STATUS_D create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field CreationDa create as date field, though DateTime requested.
  ogr_write(
/opt/anaconda3/envs/spatialdata/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field EditDate create as date field, though DateTime requested.
  ogr_write(
